### Import

In [1]:
import pubmed

### Download Neurosurgery articles

In [2]:
db = 'pubmed'
query = 'text classification neural network'
query0 = 'neurosurgery'
query1 = '(neurosurg*) AND (complicat*[Text Word])'
query2 = '("Glioma/surgery"[Mesh] OR "Glioma/therapy"[Mesh])'
query3 = '("Neurosurgery"[Mesh] OR "Neurosurgical Procedures"[Mesh]) AND "Postoperative Complications"[Mesh]'
query4 = '("Neurosurgery"[Mesh] OR "Neurosurgical Procedures"[Mesh]) AND complicat*[Text Word]'

In [3]:
article_ids = pubmed.download_all_articles(query=query, db=db, refresh=False, cache=True)

241 articles found in pubmed with query specified.


### Generate dataset

In [5]:
items = pubmed.generate_dataset(db, article_ids)
items.head()

,pmid,pmc,pii,mid,doi,elocation_id,language,title,authors,affiliations,...,publisher_location,publisher_nlm_id,publisher_issn_linking,abstract_len,abstract,copyright,mesh_quals_major,mesh_quals_minor,mesh_descriptors,keywords
0,32818665,None,S1532-0464(20)30167-2,None,10.1016/j.jbi.2020.103539,S1532-0464(20)30167-2,eng,Comparing deep learning architectures for sent...,Colón-Ruiz Cristóbal; Segura-Bedmar Isabel,"Computer Science Department, University Carlos...",...,United States,100970413,1532-0464,1899,"Since the turn of the century, as millions of ...",Copyright © 2020. Published by Elsevier Inc.,None,None,None,Bidirectional Encoder Representations from Tra...
1,32814201,None,S1532-0464(20)30169-6,None,10.1016/j.jbi.2020.103541,S1532-0464(20)30169-6,eng,A Corpus-Driven Standardization Framework for ...,Peterson Kevin J; Jiang Guoqian; Liu Hongfang,"Department of Information Technology, Mayo Cli...",...,United States,100970413,1532-0464,1739,Free-text problem descriptions are brief expla...,Copyright © 2020. Published by Elsevier Inc.,None,None,None,Deep Learning (D000077321); Health Information...
2,32807454,None,S1052-3057(20)30460-2,None,10.1016/j.jstrokecerebrovasdis.2020.105042,S1052-3057(20)30460-2,eng,Toward automatic evaluation of medical abstrac...,Fischer Igor; Steiger Hans-Jakob,"Division of Informatics and Statistics, Depart...",...,United States,9111633,1052-3057,2092,BACKGROUND: Text mining with automatic extract...,Copyright © 2020 Elsevier Inc. All rights rese...,None,None,None,Artificial neural network; Machine learning; P...
3,32765261,PMC7378532,None,None,10.3389/fphar.2020.01028,10.3389/fphar.2020.01028,eng,Artificial Intelligence in Pharmacoepidemiolog...,Sessa Maurizio; Khan Abdul Rauf; Liang David; ...,"Department of Drug Design and Pharmacology, Un...",...,Switzerland,101548923,1663-9812,1496,Aim: To perform a systematic review on the app...,"Copyright © 2020 Sessa, Khan, Liang, Andersen ...",None,None,None,artificial intelligence; deep learning; machin...
4,32764354,None,s20164369,None,10.3390/s20164369,E4369,eng,Earthquake Probability Assessment for the Indi...,Jena Ratiranjan; Pradhan Biswajeet; Al-Amri Ab...,Center for Advanced Modeling and Geospatial In...,...,Switzerland,101204366,1424-8220,2212,Earthquake prediction is a popular topic among...,None,None,None,None,GIS; Indian subcontinent; deep learning; earth...


In [6]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 240
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   pmid                    241 non-null    object
 1   pmc                     99 non-null     object
 2   pii                     146 non-null    object
 3   mid                     21 non-null     object
 4   doi                     232 non-null    object
 5   elocation_id            211 non-null    object
 6   language                241 non-null    object
 7   title                   241 non-null    object
 8   authors                 241 non-null    object
 9   affiliations            241 non-null    object
 10  article_type            241 non-null    object
 11  publication_type        241 non-null    object
 12  journal_title           241 non-null    object
 13  volume                  231 non-null    object
 14  issue                   153 non-null    object
 15  pages 

In [33]:
items.to_csv('database/{0}_queryX.csv'.format(db), sep='|')
items.to_excel('database/{0}_queryX.xlsx'.format(db))

## MySQL tests

In [8]:
import pubmed
item = {}
filename = 'pubmed/29368597'

root = pubmed.get_element_tree(filename)
item = pubmed.parse_element_tree_pubmed(root)

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from config import conn_string_pubmed
from models.pubmed_model import PubmedArticle, BasePubmed

engine = create_engine(conn_string_pubmed)
BasePubmed.metadata.bind = engine
DBSession = sessionmaker(bind=engine)

session = DBSession()

### Add

In [9]:
article = PubmedArticle(item)
session.add(article) 
session.commit()

### Read

In [7]:
results = session.query(PubmedArticle).all()

In [8]:
result = session.query(PubmedArticle).filter_by(pmid=18263039).first()
result.to_dict()

{'pmid': 18263039,
 'pmc': None,
 'pii': None,
 'mid': None,
 'doi': '10.1109/3477.485888',
 'elocation_id': None,
 'language': 'eng',
 'title': 'A dynamic gesture recognition system for the Korean sign language (KSL).',
 'authors': 'Kim J S; Jang W; Bien Z',
 'affiliations': 'Dept. of Electr. Eng., Korea Adv. Inst. of Sci. & Technol., Taejon. // Not defined // Not defined',
 'article_type': 'pubmed_article',
 'publication_type': 'Journal Article',
 'journal_title': 'IEEE transactions on systems, man, and cybernetics. Part B, Cybernetics : a publication of the IEEE Systems, Man, and Cybernetics Society',
 'volume': '26',
 'issue': '2',
 'pages': '354-9',
 'pub_date': '1996',
 'issn_electronic': None,
 'issn_print': '1083-4419',
 'journal_iso_abbr': 'IEEE Trans Syst Man Cybern B Cybern',
 'publisher_name': 'IEEE Trans Syst Man Cybern B Cybern',
 'publisher_location': 'United States',
 'publisher_nlm_id': '9890044',
 'publisher_issn_linking': '1083-4419',
 'abstract_len': 571,
 'abstract

In [10]:
print(result)

[18263039] Title: A dynamic gesture recognition system for the Korean sign language (KSL).


### Update

In [34]:
item = session.query(PubmedArticle).filter_by(pmid=18263039).first()
item.pmc = 'PMCID888'
session.commit()

### Delete

In [5]:
article = session.query(PubmedArticle).filter_by(pmid=29368597).first()
session.delete(article)
session.commit()

In [8]:
articles = session.query(PubmedArticle).all()
for article in articles:
    session.delete(article)
session.commit()

In [11]:
session.close()